In [1]:
import pandas as pd

/home/jaehun/.local/share/virtualenvs/bpr-numba-F_aeijxL/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jaehun/.local/share/virtualenvs/bpr-numba-F_aeijxL/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data_fn = '/home/jaehun/Downloads/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv'

with open(data_fn) as f:
    D = [line.replace('\n', '').split('\t') for line in f.readlines()]

In [11]:
uniq_tracks = {v:k for k, v in enumerate(list(set([line[-2] for line in D if line[-2] != ''])))}
uniq_usrs = {v:k for k, v in enumerate(list(set([line[0] for line in D])))}
ix2tracks = {v:k for k, v in uniq_tracks.items()}
ix2users = {v:k for k, v in uniq_usrs.items()}

In [4]:
res = [(uniq_usrs[d[0]], uniq_tracks[d[-2]]) for d in D if d[-2] != '']

In [5]:
del D

## Convert to DF

In [ ]:
res = pd.DataFrame(res)
res.loc[:, 2] = 1
res.columns = ['user', 'item', 'value']
res = res.groupby(['user', 'item']).sum().reset_index()
data = res

## Popularity based filtering (Densifying)

In [7]:
print('Before filtering: ', data.shape)
j = 0
d = 1
K_user = 25
K_item = 25
while d > 0:
    d_ = data.shape[0]
    user_size = data.groupby('user').size()
    data_ = data[data['user'].isin(user_size[user_size > K_user].index)]
    item_size = data_.groupby('item').size()
    data = data_[data_['item'].isin(item_size[item_size > K_item].index)]
    d = d_ - data.shape[0]
    print('Iteration {:d}:'.format(j), data.shape)
    j += 1

Before filtering:  (3963190, 3)
Iteration 0: (1309772, 3)
Iteration 1: (1309267, 3)
Iteration 2: (1309267, 3)


## Reindexing

In [22]:
data.loc[:, 'user'] = data['user'].map(ix2users)
data.loc[:, 'item'] = data['item'].map(ix2tracks)

# get new id
uniq_users = {v:k for k, v in enumerate(data['user'].unique())}
uniq_items = {v:k for k, v in enumerate(data['item'].unique())}

data.loc[:, 'user'] = data['user'].map(uniq_users)
data.loc[:, 'item'] = data['item'].map(uniq_items)

## Saving

In [25]:
data.to_csv('../bpr-numba/lastfm.subset.triplet', header=None, index=None)
with open('../bpr-numba/lastfm.subset.item.hash', 'w') as f:
    for track_id, ix in uniq_items.items():
        f.write('{}\t{:d}\n'.format(track_id, ix))
with open('../bpr-numba/lastfm.subset.user.hash', 'w') as f:
    for user_id, ix in uniq_users.items():
        f.write('{}\t{:d}\n'.format(user_id, ix))